Create a thiophene structure and run Gaussian and LAMMPS on a remote resource 

Import some standard python modules

In [1]:
import os,sys

This will use the resource creating in the resource.ipynb exmaple. This can be either a local  or remote  as long as the specified resource has 

In [2]:
from streamm import *

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
res_tag = 'local'  # Change this to remote to run the calculations remotely 
res_local = resource.Resource(res_tag)
res_local.load_json()

In [ ]:
res_local.make_dir()

In [4]:
peregrine = resource.Resource('peregrine')
peregrine.load_json()

Reading in resource json file res_peregrine.json 


In [ ]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_local.dir['home']
peregrine.dir['launch'] = res_local.dir['launch']
peregrine.dir['templates'] = '/Users/tkemper/Development/streamm_ext/streamm-tools/templates'

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [ ]:
peregrine.dump_json()

In [6]:
bbTh = buildingblock.Container('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = buildingblock.BBatom(symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

Write an xyz file that can be viewed in viewer, such as VMD or Avogadro

In [7]:
bbTh.write_xyz()

Create a gaussian calculation 

In [8]:
gaus_i = gaussian.Gaussian('th_gaus')

Add thiophene structure to calculation

In [9]:
gaus_i.set_strucC(bbTh)

Set the resource. This gives the calculation information on the location of files and if files need to coppied to a remote computer

In [10]:
gaus_i.set_resource(peregrine)

In [11]:
print os.getcwd()

/Users/tkemper/Development/streamm_ext/streamm-tools/examples


Make directories

In [12]:
gaus_i.make_dir()

Copy templates to scratch

In [13]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
gaus_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates run gaussian_peregrine.pbs templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/gaussian_peregrine.pbs to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/th_gaus/gaussian_peregrine.pbs 


In [14]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'launch'
gaus_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates com gaussian.com templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/gaussian.com to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/th_gaus/gaussian.com 


In [15]:
print gaus_i.files[file_type]

{'com': 'gaussian.com', 'run': 'gaussian_peregrine.pbs'}


Change to scratch directory where files are 

In [16]:
os.chdir(gaus_i.dir['launch'])

Load templates as strings

In [17]:
gaus_i.load_str('templates','com')

In [18]:
gaus_i.load_str('templates','run')

In [19]:
print gaus_i.str

{'run': '#!/bin/bash\n#PBS -A <allocation>\n#PBS -l walltime=<walltime>:00:00 # WALLTIME limit\n#PBS -l nodes=<nodes>:ppn=<ppn> # Number of nodes, put 16 processes on each\n#PBS -l pmem=1500\n##PBS -M <email> \n#PBS -m e\n#PBS -N <tag>\n#PBS -q <queue>\n#PBS -l feature=<feature>\n\nNPROC=<nproc>\n\nscratch_dir=<scratch_dir>\ncd $scratch_dir\n\nmodule purge;\nmodule load openmpi-gcc/1.7.3-4.8.2 python/2.7.8\n# module load mdanalysis/0.8.1\n\n# Load Gaussian module to set environment\nmodule load gaussian/g09_D.01\n\nSCRATCH=<scratch_dir>/GAUSSIAN-<tag>.$PBS_JOBID.src\nSCRATCH2=/dev/shm\n\nif [ -d $SCRATCH ]\nthen\n   rm -rf $SCRATCH\nfi\nmkdir $SCRATCH\n\n\nexport GAUSS_SCRDIR=$SCRATCH2\nls -lah /dev/shm\nrm -rf /dev/shm/*\n\necho "%NoSave" > header.text\necho "%RWF=$SCRATCH2/,1500MB,$SCRATCH/,-1" >> header.text\n\ncp <input_com> body.text\ncat header.text  body.text  >  <input_com> \n\ng09 < <input_com> >&  <tag>.log\n\nformchk <tag>.chk\n', 'com': '%chk=<tag>.chk\n#P <commands> \n\n<t

Set calculation properties specified in .com template

In [20]:
gaus_i.properties['commands'] = 'HF/3-21G SP'
gaus_i.properties['charge'] = 0
gaus_i.properties['spin_mult'] = 1
gaus_i.properties['coord'] = gaus_i.strucC.write_coord()

Write .com file and record it in calculation's input files

In [21]:
gaus_i.replacewrite_prop('com','input','com','%s.com'%(gaus_i.tag))

Set calculation properties specified in .sh template

In [22]:
gaus_i.properties['input_com'] = gaus_i.files['input']['com']

In [23]:
print gaus_i.dir['scratch']
gaus_i.properties['scratch_dir'] = gaus_i.dir['scratch']

/scratch/tkemper/th_gaus/


Write run script for running calculation

In [24]:
gaus_i.replacewrite_prop('run','scripts','run','%s.sh'%(gaus_i.tag))

Add log file to output files

In [25]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(gaus_i.tag)
gaus_i.add_file(file_type,file_key,file_name)

In [26]:
file_type = 'output'
file_key = 'fchk'
file_name = "%s.fchk"%(gaus_i.tag)
gaus_i.add_file(file_type,file_key,file_name)

Checkpoint calculation by outputing json file

In [27]:
os.chdir(gaus_i.dir['home'])
gaus_i.dump_json()
os.chdir(gaus_i.dir['launch'])

In [ ]:
gaus_i.push()

Run calculation

In [ ]:
gaus_i.run()

Check the status of the calculation

In [28]:
gaus_i.check()
print gaus_i.meta['status']

finished


Analyize output of calculation

In [29]:
gaus_i.analysis()

Set charges of structure to mulliken charges from the calculation

In [30]:
gaus_i.strucC.properties['charge'] = 0.0 
for p,pt in gaus_i.strucC.particles.iteritems(): # properties["charge"] 
    pt.properties["charge"] = round(pt.properties["mulliken"],3)
    gaus_i.strucC.properties['charge']  += pt.properties["charge"]
    print pt.properties["charge"]

-0.71
-0.2
-0.2
-0.71
0.766
0.28
0.247
0.247
0.28


Check that rounding has not affected the total charge

In [31]:
print "Total charge %f "%(gaus_i.strucC.properties['charge'])

Total charge 0.000000 


Store output 

In [ ]:
gaus_i.store()

Check that that data can be pulled into the local scratch directory 

In [ ]:
gaus_i.pull()

Guassian calculation of thiophene is finished, return to home directory of calculation

In [32]:
os.chdir(gaus_i.dir['home'])

In [ ]:
gaus_i.dump_json()

Start lammps calculation

In [33]:
lmp_i = lammps.LAMMPS('th_lmp')

In [34]:
lmp_i.set_resource(peregrine)

In [60]:
print lmp_i.dir['scratch']

/scratch/tkemper/th_lmp/


In [61]:
lmp_i.properties['scratch'] = lmp_i.dir['scratch']

In [35]:
lmp_i.make_dir()

Convert the structure from the gaussian output to the lammmps calculation

In [36]:
lmp_i.convert_gaussian(gaus_i)

Set box size

In [37]:
lmp_i.strucC.lat_cubic(25.0)

Find nieghbors bassed on distances and covalent radii

In [38]:
lmp_i.strucC.bonded_nblist.guess_nblist(lmp_i.strucC.lat,lmp_i.strucC.particles,lmp_i.strucC.positions,"cov_radii",radii_buffer=1.25)

Read in force-field parameters

In [39]:
lmp_i.read_param("conj.param")

Guess force-field atom types 

In [40]:
lmp_i.strucC.guess_oplsa()

Set bonded interactions

In [41]:
lmp_i.strucC.bonded_bonds()
lmp_i.strucC.bonded_angles()
lmp_i.strucC.bonded_dih()

Clean up force-field parameters so only the parameters for the interactions in the molecule are included

In [42]:
lmp_i.set_ffparam()

In [43]:
print peregrine.dir['templates'] 

/Users/tkemper/Development/streamm_ext/streamm-tools/templates


Read in templates

In [44]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_sp.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates in lammps_sp.in templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/lammps_sp.in to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/th_lmp/lammps_sp.in 


In [45]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates run lammps_peregrine.pbs templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/lammps_peregrine.pbs to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/th_lmp/lammps_peregrine.pbs 


In [46]:
print lmp_i.files[file_type]

{'run': 'lammps_peregrine.pbs', 'in': 'lammps_sp.in'}


Write input files in scratch directory

In [47]:
os.chdir(lmp_i.dir['launch'])

In [48]:
lmp_i.load_str('templates','run')

In [49]:
lmp_i.load_str('templates','in')

In [50]:
lmp_i.write_data()

In [62]:
lmp_i.replacewrite_prop('in','input','in','%s.in'%(lmp_i.tag))

In [63]:
lmp_i.properties['input_in'] = lmp_i.files['input']['in']

In [64]:
lmp_i.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_i.tag))

Add output log file to output files

In [65]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_i.tag)
lmp_i.add_file(file_type,file_key,file_name)

In [66]:
os.chdir(lmp_i.dir['home'])
lmp_i.dump_json()
os.chdir(lmp_i.dir['launch'])

In [67]:
lmp_i.push()

 Resource type ssh 
runnning push function in /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/th_lmp 
Compressing and copying input files to scratch directory 
Compressing and copying templates files to scratch directory 
Compressing and copying scripts files to scratch directory 


Run calculation

In [68]:
lmp_i.run()

Calculation with status written 
Resource type ssh 
Executing run command ssh tkemper@peregrine.hpc.nrel.gov ' cd /scratch/tkemper/th_lmp/ ; qsub th_lmp.sh '  


In [69]:
lmp_i.check()
print lmp_i.meta['status']

finished


In [70]:
lmp_i.analysis()
print lmp_i.meta['status']

finished


In [71]:
print lmp_i.properties['run_cnt']
run_i = lmp_i.properties['run_list'][0]
print run_i.properties['timestep'],run_i.timeseries.keys()
print run_i.timeseries['step']
print run_i.timeseries['toteng']
print run_i.timeseries['poteng']

1
0.5 ['e_tail', 'e_dihed', 'temp', 'e_long', 'e_bond', 'e_coul', 'volume', 'step', 'e_impro', 'e_vdwl', 'e_angle', 'press', 'toteng', 'lz', 'poteng', 'lx', 'ly']
[0.0]
[30.77175]
[30.77175]


In [72]:
del lmp_i,gaus_i